<a href="https://colab.research.google.com/github/wesslen/llm-examples/blob/main/notebooks/dspy_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dspy

lm = dspy.LM("gemini-2.5-flash-lite")
dspy.configure(lm = lm)

lm("What is 2+2")

rag = dspy.ChainOfThought("context, question -> answer")
r = rag(context = "it's 12pm", question = "What time is it?")

r.answer

# DSPy tools to make async - will help

with dspy.context(lm = lm):
  r = rag(context = "it's 12pm", question = "What time is it?")